In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df_interactions = pd.read_csv('../data/interactions.csv')

In [3]:
df_interactions.count()

user_id      1124532
recipe_id    1124532
rating       1124532
dtype: int64

In [136]:
df_recipe_popularity = pd.read_csv('../data/recipe_popularity.csv')

In [ ]:
df_recipe_popularity.count()

In [77]:
df_interactions.groupby(["user_id"]).count().sort_values("rating", ascending=False).reset_index().head(100)

,user_id,recipe_id,rating
0,6043f4dcb8458600693a374d,7665,7665
1,60cbbe3bed6fcb006949d796,5587,5587
2,60cbbe00348edc006a22c826,4621,4621
3,60cbbdb8ed6fcb006949d771,4073,4073
4,60cbbd7c7f82d100681b3386,3912,3912
...,...,...,...
95,2875f5879dad4118aff113d0,1010,1010
96,64bbfd442dd8434f8bd71234,994,994
97,05f01b5ce879451c9deb2c37,991,991
98,ed5ae65ae48d424a8bfc4332,982,982


In [138]:
df_top_reviewers = df_interactions.groupby(["user_id"]).count().sort_values("rating", ascending=False).reset_index()["user_id"]
#df_top_reviewers = df_top_reviewers.head(250)
df_bottom_reviewers = df_interactions.groupby(["user_id"]).count().sort_values("rating", ascending=True).reset_index()["user_id"]
#df_bottom_reviewers = df_bottom_reviewers.head(5000)

In [139]:
df_reviewers = pd.concat([df_top_reviewers, df_bottom_reviewers]).drop_duplicates()

In [140]:
df_reviews = df_interactions.loc[df_interactions["user_id"].isin(df_reviewers)]
df_reviews = df_reviews.reset_index(drop=True)

In [141]:
df_reviews.count()

user_id      1124532
recipe_id    1124532
rating       1124532
date         1124532
dtype: int64

In [142]:
df_joined = df_reviews.merge(df_recipe_popularity[["recipe_id", "rating_count", "rating_sum", "rating_avg", "rating_weighted_avg", "rank"]], on=["recipe_id"], how="left")

In [143]:
df_joined.count()

user_id                1124532
recipe_id              1124532
rating                 1124532
date                   1124532
rating_count           1124532
rating_sum             1124532
rating_avg             1124532
rating_weighted_avg    1124532
rank                   1124532
dtype: int64

In [145]:
df_joined.head()

,user_id,recipe_id,rating,date,rating_count,rating_sum,rating_avg,rating_weighted_avg,rank
0,auth0|37a18c3939764ac9b4d72bda,b355f2ab-a8e3-4c69-833c-3ac31f5b0022,4,2017-09-04,701,3013,4.298146,4.303293,214372
1,auth0|794e065d91d74983bbec1e2d,5d063835-3353-4072-8742-754a2b6931bf,4,2007-02-05,4,17,4.250000,4.342316,176924
2,auth0|b17d1cd98ba34cb89cf03827,4fd380ff-2998-418b-aa0f-7bf262452e5b,5,2010-03-06,3,10,3.333333,4.311412,206756
3,auth0|bc88bec56fa54115ae355987,b4a5a1b7-3015-4a5b-ae59-9f79fb6d7bb5,5,2008-04-17,51,243,4.764706,4.505832,619
4,auth0|bfa42caef31f42f2871ebd8d,4f128c81-e2ab-449c-91ca-194ab31b2d6e,5,2009-04-16,14,64,4.571429,4.379190,22109


In [146]:
train, test = train_test_split(df_joined, test_size=0.2)

In [147]:
train.to_csv('../data/ml_dotnet_interactions_train.csv', index=False)
test.to_csv('../data/ml_dotnet_interactions_test.csv', index=False)